In [1]:
import pandas as pd
pd.set_option('display.max_rows', None) # Print all rows

In [2]:
# FIX SSL: CERTIFICATE_VERIFY_FAILED with Python3
# https://stackoverflow.com/a/49174340
import ssl

ssl._create_default_https_context = ssl._create_unverified_context

In [3]:
url = 'https://www.juntadeandalucia.es/institutodeestadisticaycartografia/badea/stpivot/stpivot/Print?cube=013d884c-cfc5-4c21-9e78-e26225682afd&type=3&foto=si&ejecutaDesde=&codConsulta=38668&consTipoVisua=JP'
fields = ['Lugar de residencia', 'Medida', 'Valor']
df = pd.read_csv(url, delimiter=';', usecols=fields, decimal=',')

In [4]:
df[df['Lugar de residencia'] == 'Huelva']

,Lugar de residencia,Medida,Valor
0,Huelva,Población,513170
1,Huelva,Confirmados PDIA,21995
2,Huelva,Confirmados PDIA 14 días,5432
3,Huelva,Tasa PDIA 14 días,"1058,5186195607694"
4,Huelva,Confirmados PDIA 7 días,2173
5,Huelva,Total Confirmados,22222
6,Huelva,Curados,11741
7,Huelva,Fallecidos,189


In [5]:
df['Valor'] = pd.to_numeric(df['Valor'],errors='coerce')

In [6]:
dfAnterior = pd.read_csv('covid_huelva.csv', index_col='Lugar de residencia')

In [7]:
# Confirmados PCR provincia Huelva
p = df[df['Lugar de residencia'] == 'Huelva'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en la provincia de Huelva: ', p)

Confirmados PCR en la provincia de Huelva:  21995.0


In [8]:
# Confirmados PCR Huelva capitalpd
c = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA']['Valor'].values[0]
print('Confirmados PCR en Huelva capital: ', c)

Confirmados PCR en Huelva capital:  5848.0


In [9]:
c7 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 7 días']['Valor'].values[0]
c14 = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Confirmados PDIA 14 días']['Valor'].values[0]
p = df[df['Lugar de residencia'] == 'Huelva (capital)'][df['Medida'] == 'Población']['Valor'].values[0]

In [10]:
print('Huelva (capital)\n')
#print('Confirmados PCR 7 días x 1,000 habitantes: ', f"{c7/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c7:.0f}", 'personas en los últimos 7 días', '\n')
#print('Confirmados PCR 14 días x 1,000 habitantes: ', f"{c14/(p/1000):.5f}")
print('Un positivo PCR cada', f"{p/c14:.0f}", 'personas en los últimos 14 días')

Huelva (capital)

Un positivo PCR cada 281 personas en los últimos 7 días 

Un positivo PCR cada 110 personas en los últimos 14 días


In [11]:
filtro = ((df['Medida'] == 'Confirmados PDIA')| (df['Medida'] == 'Confirmados PDIA 7 días') | (df['Medida'] == 'Población' ) | (df['Medida'] == 'Confirmados PDIA 14 días')) & (df['Valor'] > 0)
tabla = pd.pivot_table(df.loc[filtro], values='Valor', index='Lugar de residencia', columns='Medida').dropna()

In [12]:
# Reorder columns
tabla = tabla[['Confirmados PDIA', 'Confirmados PDIA 7 días', 'Confirmados PDIA 14 días', 'Población']]

In [13]:
tabla['tasa7'] = tabla['Confirmados PDIA 7 días']/tabla['Población']*100000
tabla['tasa14'] = tabla['Confirmados PDIA 14 días']/tabla['Población']*100000

In [14]:
tabla['Diferencia PDIA'] = tabla['Confirmados PDIA'] - dfAnterior['Confirmados PDIA']

In [15]:
# Nuevos casos desde la última captura
tabla[tabla['Diferencia PDIA'] != 0].sort_values('Diferencia PDIA', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Huelva,21995.0,2173.0,5432.0,513170.0,423.446421,1058.518620,370.0
Huelva-Costa,13339.0,1283.0,3382.0,289548.0,443.104425,1168.027408,193.0
Condado-Campiña,6057.0,655.0,1545.0,156231.0,419.250981,988.920253,133.0
Huelva (capital),5848.0,511.0,1308.0,143837.0,355.263249,909.362681,68.0
Sierra de Huelva-Andévalo Central,2230.0,148.0,386.0,67391.0,219.613895,572.776780,27.0
Lepe,1762.0,183.0,440.0,27880.0,656.384505,1578.192253,23.0
Isla Cristina,1809.0,152.0,478.0,21393.0,710.512785,2234.375730,21.0
Almonte,695.0,77.0,159.0,24507.0,314.195944,648.794222,20.0
Ayamonte,1032.0,135.0,379.0,21104.0,639.689158,1795.868082,20.0


In [16]:
if tabla[tabla['Diferencia PDIA'] != 0]['Diferencia PDIA'].sum() > 0:
  # Save to csv
  tabla.to_csv('covid_huelva.csv', encoding='utf-8', header=True, index=True)  

In [17]:
tabla.sort_values('tasa14', ascending=False)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA
Lugar de residencia,,,,,,,
Villalba del Alcor,416.0,34.0,126.0,3366.0,1010.101010,3743.315508,6.0
Berrocal,16.0,2.0,10.0,303.0,660.066007,3300.330033,2.0
Paymogo,66.0,15.0,35.0,1162.0,1290.877797,3012.048193,6.0
San Bartolomé de la Torre,179.0,38.0,90.0,3761.0,1010.369583,2392.980590,7.0
Isla Cristina,1809.0,152.0,478.0,21393.0,710.512785,2234.375730,21.0
Cabezas Rubias,26.0,6.0,13.0,706.0,849.858357,1841.359773,0.0
Ayamonte,1032.0,135.0,379.0,21104.0,639.689158,1795.868082,20.0
Palma del Condado (La),911.0,81.0,188.0,10801.0,749.930562,1740.579576,13.0
Lepe,1762.0,183.0,440.0,27880.0,656.384505,1578.192253,23.0


In [18]:
# Peso de la tasa de la última semana en relación con la tasa a 14 días 
# Mejor cuanto menor. Valor 1 significa que no había casos antes de 7 días.
tabla['7/14'] = tabla['tasa7'] / tabla['tasa14']
tabla.fillna(0).sort_values('7/14', ascending=True)

Medida,Confirmados PDIA,Confirmados PDIA 7 días,Confirmados PDIA 14 días,Población,tasa7,tasa14,Diferencia PDIA,7/14
Lugar de residencia,,,,,,,,
Higuera de la Sierra,29.0,1.0,8.0,1291.0,77.459334,619.674671,0.0,0.125000
Berrocal,16.0,2.0,10.0,303.0,660.066007,3300.330033,2.0,0.200000
Manzanilla,79.0,5.0,20.0,2118.0,236.071766,944.287063,0.0,0.250000
Aracena,444.0,17.0,64.0,8255.0,205.935796,775.287704,2.0,0.265625
Villalba del Alcor,416.0,34.0,126.0,3366.0,1010.101010,3743.315508,6.0,0.269841
Beas,136.0,11.0,39.0,4341.0,253.397835,898.410504,0.0,0.282051
Puebla de Guzmán,69.0,4.0,14.0,3092.0,129.366106,452.781371,1.0,0.285714
Isla Cristina,1809.0,152.0,478.0,21393.0,710.512785,2234.375730,21.0,0.317992
Punta Umbría,580.0,77.0,241.0,15355.0,501.465321,1569.521329,6.0,0.319502
